# Data preparation

## Setup

In [ ]:
import glob
import os

import pandas as pd

## Bitcoin

In [ ]:
# Match the path of the raw CSV
csv_path = glob.glob(os.path.expanduser("~/Downloads/bitcoin_2010-07-22*.csv"))[0]

In [ ]:
# Read raw CSV as df and show it
df_btc = pd.read_csv(csv_path)
df_btc

In [ ]:
# Reverse the order of the rows
df_btc = df_btc[::-1].copy()

In [ ]:
# Remove irrelevant columns, and rename the remaining
df_btc.drop(columns=["End", "Volume", "Market Cap"], inplace=True)
df_btc.rename(columns={"Start": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close"}, inplace=True)

In [ ]:
# Check the data type of each column
df_btc.dtypes

In [ ]:
# Check if the data type of each column is consistent across all rows
df_btc.map(type).nunique() == 1

In [ ]:
# Check for null values in each column
df_btc.isna().sum()

In [ ]:
# Check for zero values in each column
(df_btc == 0).sum()

In [ ]:
# Check if dates progress correctly (without skipped or duplicated dates)
df_btc["date"] = pd.to_datetime(df_btc["date"])
date_diff = df_btc["date"].diff().dropna()
(date_diff == pd.Timedelta(days=1)).all()

In [ ]:
# Reset index and show final df
df_btc.reset_index(drop=True, inplace=True)
df_btc

In [ ]:
# Export final df
df_btc.to_csv("../data/BTC.csv", index=False)

### Dataset basic info

In [ ]:
# First and last entries
pd.concat([df_btc.head(1), df_btc.tail(1)]).T

In [ ]:
df_btc.describe().T

In [ ]:
# How many trading days per year on average
days_per_year = df_btc[~df_btc.index.year.isin([2024, 2010])].index.year.value_counts()
days_per_year.mean().round(2)